In [1]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Computer Vision with CNNs
#
# For this exercise you will build a cats v dogs classifier
# using the Cats v Dogs dataset from TFDS.
# Be sure to use the final layer as shown 
#     (Dense, 2 neurons, softmax activation)
#
# The testing infrastructre will resize all images to 224x224 
# with 3 bytes of color depth. Make sure your input layer trains
# images to that specification, or the tests will fail.
#
# Make sure your output layer is exactly as specified here, or the 
# tests will fail.

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint

dataset_name = 'cats_vs_dogs'
# dataset = tfds.load(name=dataset_name, split=tfds.Split.TRAIN)
dataset_t = tfds.load(name=dataset_name, split='train[:80%]')
dataset_v = tfds.load(name=dataset_name, split='train[80%:]')

# for data in dataset_t.take(1):
#   print(data)

#최소최대정규화, 이미지크기(224x224)
def preprocess(data):
    # YOUR CODE HERE
    x = data['image']
    y = data['label']
    x = tf.cast(x, tf.float32)/255.    #형변환 -> 최소최대정규화 
    x = tf.image.resize(x, size=(224,224))   #이미지 크기 변환 
    return x, y


def solution_model():
    # train_dataset = dataset.map(preprocess).batch(32)
    train_dataset = dataset_t.map(preprocess).batch(32)
    valid_dataset = dataset_v.map(preprocess).batch(32)

    model = Sequential([
        Conv2D(64, (3,3), input_shape=(224,224,3), activation='relu'),
        MaxPool2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPool2D(2,2),
        Conv2D(256, (3,3), activation='relu'),
        MaxPool2D(2,2), 
        Conv2D(512, (3,3), activation='relu'),
        MaxPool2D(2,2), 
        Flatten(),
        Dropout(0.5),
        Dense(512, activation='relu'),  
        Dropout(0.5),                
        Dense(256, activation='relu'),                  
        # YOUR CODE HERE, BUT MAKE SURE YOUR LAST LAYER HAS 2 NEURONS ACTIVATED BY SOFTMAX
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    print(model.summary())
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['acc'])
    checkpoint_path = 'cat_dog_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, 
                                 save_best_only=True, monitor='val_loss',verbose=1)
    model.fit(train_dataset, validation_data=(valid_dataset), epochs=30,
              callbacks=[checkpoint])
    model.load_weights(checkpoint_path)
    return model


# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("TF3-cats-vs-dogs.h5")

# 목표 : 0.3158 


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete29M1CF/cats_vs_dogs-train.tfrecord


Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 256)       